# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import pprint
import collections.abc
#the  following aliasesrequired for gmaps.marker_layer collections.iterable to work
collections.Iterable = collections.abc.Iterable

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "../output_data/city_weather_data.csv"
weather_df = pd.read_csv(weather_csv_file)

weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,loa janan,-0.5830,117.0950,74.46,96,80,0.83,ID,1655041250
1,lebu,-37.6167,-73.6500,48.45,84,70,7.45,CL,1655041282
2,ushuaia,-54.8000,-68.3000,46.06,71,0,5.75,AR,1655041283
3,rikitea,-23.1203,-134.9692,72.82,63,67,8.88,PF,1655041283
4,inuvik,68.3499,-133.7218,34.66,93,100,8.05,CA,1655041283
...,...,...,...,...,...,...,...,...,...
577,gazojak,41.1875,61.4036,88.54,10,100,6.82,TM,1655041450
578,mariental,-24.6333,17.9667,77.54,18,0,15.37,NaN,1655041203
579,dhidhdhoo,6.8833,73.1000,82.26,76,100,12.06,MV,1655041438
580,mandalgovi,45.7625,106.2708,58.05,50,5,8.97,MN,1655041451


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
weather_df = weather_df.dropna()

# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]

# Create a Humidity heatmap layer
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100, 
                                 point_radius = 2)

fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the DataFrame to find your ideal weather condition.
ideal_weather_df = weather_df.loc[(weather_df["Max Temp"] < 80) 
                                    & (weather_df["Max Temp"] > 70) 
                                    & (weather_df["Windspeed"] < 10) 
                                    & (weather_df["Cloudiness"] == 0),:].dropna()
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
273,jinchang,38.4953,102.1739,71.26,10,0,4.27,CN,1655041363
305,silver city,32.7701,-108.2803,77.68,30,0,8.05,US,1655041371
342,greeneville,36.1632,-82.8310,78.39,67,0,1.01,US,1655041381
355,california city,35.1258,-117.9859,76.10,27,0,8.05,US,1655041385
469,payson,40.0444,-111.7321,77.41,39,0,3.44,US,1655041417
489,concarneau,47.8750,-3.9225,74.82,51,0,7.16,FR,1655041425


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Max Temp"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Max Temp,Hotel Name
273,jinchang,CN,38.4953,102.1739,71.26,
305,silver city,US,32.7701,-108.2803,77.68,
342,greeneville,US,36.1632,-82.8310,78.39,
355,california city,US,35.1258,-117.9859,76.10,
469,payson,US,40.0444,-111.7321,77.41,
489,concarneau,FR,47.8750,-3.9225,74.82,


In [6]:
#Set parameters to search for hotels with 5000 meters
#Hit the Google Places API for each city's coordinates.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through cities
for index, row in hotel_df.iterrows():
    # get lat, lng from dataframe
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while keeping original params in place
    params["location"] = f"{lat},{lng}"
    
    # Create base url to search nearby hotel     
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request
    print(f"Retrieving results for city: {row['City']}")
    hotel_name = requests.get(base_url, params=params).json()
    
    # extract results
    hotel_name = hotel_name['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name[0]['name']
        print(f"Closest hotel is {hotel_name[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = hotel_name[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Display hotel names on dataframe
hotel_df

Retrieving results for city: jinchang
Closest hotel is Mingshihui Jinchang International Hotel.
------------
Retrieving results for city: silver city
Closest hotel is Copper Manor Motel.
------------
Retrieving results for city: greeneville
Closest hotel is General Morgan Inn and Conference Center.
------------
Retrieving results for city: california city
Closest hotel is Best Western California City Inn & Suites.
------------
Retrieving results for city: payson
Closest hotel is Quality Inn Payson I-15.
------------
Retrieving results for city: concarneau
Closest hotel is Brit Hotel de L' Océan.
------------


,City,Country,Lat,Lng,Max Temp,Hotel Name
273,jinchang,CN,38.4953,102.1739,71.26,Mingshihui Jinchang International Hotel
305,silver city,US,32.7701,-108.2803,77.68,Copper Manor Motel
342,greeneville,US,36.1632,-82.8310,78.39,General Morgan Inn and Conference Center
355,california city,US,35.1258,-117.9859,76.10,Best Western California City Inn & Suites
469,payson,US,40.0444,-111.7321,77.41,Quality Inn Payson I-15
489,concarneau,FR,47.8750,-3.9225,74.82,Brit Hotel de L' Océan


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [9]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))